In [2]:
# dependencies
import pandas as pd
import numpy as np
import os
import csv

In [3]:
# File to load
schools_csv = os.path.join("../Resources/schools_complete.csv")
students_csv = os.path.join("../Resources/students_complete.csv")

In [4]:
schools_pd = pd.read_csv(schools_csv)
students_pd = pd.read_csv(students_csv)

In [5]:
schools_pd.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [6]:
students_pd.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [7]:
students_merged_pd = pd.merge(students_pd, schools_pd, how="left", on=["school_name", "school_name"])
students_merged_pd.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [8]:
# Add 0/1 indicators for passing math, reading and overall (so that they can be summed later on)
students_merged_pd['passed_math'] = np.where(students_merged_pd['math_score']>=70, 1, 0)
students_merged_pd['passed_reading'] = np.where(students_merged_pd['reading_score']>=70, 1, 0)
students_merged_pd['passed_overall'] = np.where(students_merged_pd['passed_math'] + students_merged_pd['passed_reading'] == 2, 1, 0)


In [9]:
# District Summary

# Total Schools
total_schools = students_merged_pd['School ID'].value_counts().count()

# Total Students
total_students = students_merged_pd['Student ID'].value_counts().count()

# Total Budget
total_budget = schools_pd['budget'].sum()

# Average Math Score
avg_math_score = students_merged_pd['math_score'].mean()

# Average Reading Score
avg_reading_score = students_merged_pd['reading_score'].mean()

# % Passing Math
percent_pass_math = 100 * students_merged_pd['passed_math'].sum() / total_students

# % Passing Reading
percent_pass_reading = 100 * students_merged_pd['passed_reading'].sum() / total_students

# % Overall Passing
percent_pass_overall = 100 * students_merged_pd['passed_overall'].sum() / total_students

district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
                                    "Total Students": [total_students],
                                    "Total Budget": [total_budget],
                                    "Average Math Score": [avg_math_score],
                                    "Average Reading Score": [avg_reading_score],
                                    "% Passing Math": [percent_pass_math],
                                    "% Passing Reading": [percent_pass_reading],
                                    "% Overall Passing": [percent_pass_overall]})

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [10]:
# School Summary

# group by school (include budget since it should not be added over student rows)
students_grouped_by_school_pd = students_merged_pd.groupby(['school_name','type','budget','size']).agg({'Student ID': ['count'],'math_score': ['mean'], 'reading_score': ['mean'], 'passed_math': ['sum'], 'passed_reading': ['sum'], 'passed_overall': ['sum']}).reset_index()
students_grouped_by_school_pd.columns = students_grouped_by_school_pd.columns.droplevel(1)

# rename the dataframe columns that will be kept in the output table
students_grouped_by_school_pd = students_grouped_by_school_pd.rename(columns = {'school_name':'School Name','type':'School Type','budget':'Total School Budget','Student ID':'Total Students','math_score':'Average Math Score','reading_score':'Average Reading Score'})

# add the per student budget
students_grouped_by_school_pd['Per Student Budget'] = students_grouped_by_school_pd['Total School Budget'] /students_grouped_by_school_pd['Total Students']

# add the reading, math, and overall pass percentages
students_grouped_by_school_pd['% Passing Math'] = 100 * students_grouped_by_school_pd['passed_math'] / students_grouped_by_school_pd['Total Students']
students_grouped_by_school_pd['% Passing Reading'] = 100 * students_grouped_by_school_pd['passed_reading'] / students_grouped_by_school_pd['Total Students']
students_grouped_by_school_pd['% Overall Passing'] = 100 * students_grouped_by_school_pd['passed_overall'] / students_grouped_by_school_pd['Total Students']

# set the dataframe index to be the school name, not 0,1,2...
students_grouped_by_school_pd = students_grouped_by_school_pd.set_index(['School Name'])

# select the desired output columns and set formats
output = students_grouped_by_school_pd[['School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing']]

output


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [11]:
# top performing
students_grouped_by_school_pd.sort_values(by = ['% Overall Passing'], ascending=False).head(5)

,School Type,Total School Budget,size,Total Students,Average Math Score,Average Reading Score,passed_math,passed_reading,passed_overall,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,,,,,
Cabrera High School,Charter,1081356,1858,1858,83.061895,83.975780,1749,1803,1697,582.0,94.133477,97.039828,91.334769
Thomas High School,Charter,1043130,1635,1635,83.418349,83.848930,1525,1591,1487,638.0,93.272171,97.308869,90.948012
Griffin High School,Charter,917500,1468,1468,83.351499,83.816757,1371,1426,1330,625.0,93.392371,97.138965,90.599455
Wilson High School,Charter,1319574,2283,2283,83.274201,83.989488,2143,2204,2068,578.0,93.867718,96.539641,90.582567
Pena High School,Charter,585858,962,962,83.839917,84.044699,910,923,871,609.0,94.594595,95.945946,90.540541


In [12]:
# bottom performing
students_grouped_by_school_pd.sort_values(by = ['% Overall Passing'], ascending=True).head(5)

,School Type,Total School Budget,size,Total Students,Average Math Score,Average Reading Score,passed_math,passed_reading,passed_overall,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,,,,,
Rodriguez High School,District,2547363,3999,3999,76.842711,80.744686,2654,3208,2119,637.0,66.366592,80.220055,52.988247
Figueroa High School,District,1884411,2949,2949,76.711767,81.158020,1946,2381,1569,639.0,65.988471,80.739234,53.204476
Huang High School,District,1910635,2917,2917,76.629414,81.182722,1916,2372,1561,655.0,65.683922,81.316421,53.513884
Hernandez High School,District,3022020,4635,4635,77.289752,80.934412,3094,3748,2481,652.0,66.752967,80.862999,53.527508
Johnson High School,District,3094650,4761,4761,77.072464,80.966394,3145,3867,2549,650.0,66.057551,81.222432,53.539172


In [13]:
# math scores by grade

# group the dataset by school and grade, and average the math score 
math_by_school_and_grade_pd = students_merged_pd.groupby(['school_name','grade'])['math_score'].mean().reset_index()
math_by_school_and_grade_pd.pivot(index='school_name', columns='grade', values='math_score')


grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [14]:
# reading scores by grade

# group the dataset by school and grade, and average the reading score 
reading_by_school_and_grade_pd = students_merged_pd.groupby(['school_name','grade'])['reading_score'].mean().reset_index()

reading_by_school_and_grade_pd.pivot(index='school_name', columns='grade', values='reading_score')

# Need to figure out how to re-order columns


grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


In [15]:
# scores by school spending

#figure out 4 appropriate ranges for per student budget using quartiles
spending_quartiles = students_grouped_by_school_pd

# group the original dataframe per student buget by school using these quartiles
spending_quartiles['quartile'] = pd.qcut(students_grouped_by_school_pd['Per Student Budget'], 4, ['578 to 599','600 to 636','637 to 643','644 and up'])

# show the start values of each quartile to assign an appropriate range name instead of q1, q2, q3, q4
# quartiles = spending_quartiles.groupby(['quartile'])['Per Student Budget'].min().reset_index()
# qartiles

# need to do a weighted average (multiply the number of students 
# by the average math/reading scores then sum and divide by the quartile total)
spending_quartiles['school_total_math_scores'] = spending_quartiles['Total Students'] * spending_quartiles['Average Math Score']
spending_quartiles['school_total_reading_scores'] = spending_quartiles['Total Students'] * spending_quartiles['Average Reading Score']

spending_quartiles

#re-sum the math and reading and retest the overall pass (same way as previous section)
spending_quartiles = spending_quartiles.groupby(['quartile']).agg({'Total Students': ['sum'],'passed_math': ['sum'], 'passed_reading': ['sum'], 'passed_overall': ['sum'], 'school_total_math_scores': ['sum'],'school_total_reading_scores': ['sum']}).reset_index()
spending_quartiles.columns = spending_quartiles.columns.droplevel(1)

spending_quartiles= spending_quartiles[['quartile','Total Students','passed_math','passed_reading','passed_overall','school_total_math_scores','school_total_reading_scores']]

spending_quartiles['Average Math Score'] = spending_quartiles['school_total_math_scores'] / spending_quartiles['Total Students']
spending_quartiles['Average Reading Score'] = spending_quartiles['school_total_reading_scores'] / spending_quartiles['Total Students']
spending_quartiles['% Passing Math'] = 100 * spending_quartiles['passed_math'] / spending_quartiles['Total Students']
spending_quartiles['% Passing Reading'] = 100 * spending_quartiles['passed_reading'] / spending_quartiles['Total Students']
spending_quartiles['% Overall Passing'] = 100 * spending_quartiles['passed_overall'] / spending_quartiles['Total Students']

school_spending_output = spending_quartiles[['quartile','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing']]

school_spending_output


,quartile,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,578 to 599,83.363065,83.964039,93.702889,96.686558,90.640704
1,600 to 636,79.982873,82.312643,79.109851,88.513145,70.939239
2,637 to 643,78.050332,81.478038,71.361995,83.653734,60.293604
3,644 and up,77.058995,80.958411,66.609088,80.779963,53.667287


In [16]:
# scores by school size

#figure out 4 appropriate ranges for per student budget using quartiles
size_quartiles = students_grouped_by_school_pd

# group the original dataframe per student buget by school using these quartiles
size_quartiles['quartile'] = pd.qcut(students_grouped_by_school_pd['size'], 4, [' 427 to 1,760','1,761 to 2,738','2,739 to 3,998','3,999 and up'])
# size_quartiles

# show the start values of each quartile to assign an appropriate range name instead of q1, q2, q3, q4
# quartiles = size_quartiles.groupby(['quartile'])['size'].min().reset_index()
# quartiles

# need to do a weighted average (multiply the number of students 
# by the average math/reading scores then sum and divide by the quartile total)
size_quartiles['school_total_math_scores'] = size_quartiles['Total Students'] * size_quartiles['Average Math Score']
size_quartiles['school_total_reading_scores'] = size_quartiles['Total Students'] * size_quartiles['Average Reading Score']

# size_quartiles
# #re-sum the math and reading and retest the overall pass (same way as previous section)
size_quartiles = size_quartiles.groupby(['quartile']).agg({'Total Students': ['sum'],'passed_math': ['sum'], 'passed_reading': ['sum'], 'passed_overall': ['sum'], 'school_total_math_scores': ['sum'],'school_total_reading_scores': ['sum']}).reset_index()
size_quartiles.columns = size_quartiles.columns.droplevel(1)

size_quartiles= size_quartiles[['quartile','Total Students','passed_math','passed_reading','passed_overall','school_total_math_scores','school_total_reading_scores']]
size_quartiles


size_quartiles['Average Math Score'] = size_quartiles['school_total_math_scores'] / size_quartiles['Total Students']
size_quartiles['Average Reading Score'] = size_quartiles['school_total_reading_scores'] / size_quartiles['Total Students']
size_quartiles['% Passing Math'] = 100 * size_quartiles['passed_math'] / size_quartiles['Total Students']
size_quartiles['% Passing Reading'] = 100 * size_quartiles['passed_reading'] / size_quartiles['Total Students']
size_quartiles['% Overall Passing'] = 100 * size_quartiles['passed_overall'] / size_quartiles['Total Students']

school_size_output = size_quartiles[['quartile','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing']]

school_size_output

,quartile,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,"427 to 1,760",83.523375,83.877115,93.521817,96.861086,90.583259
1,"1,761 to 2,738",83.337834,83.917814,93.806803,96.520384,90.547910
2,"2,739 to 3,998",76.808251,81.035328,66.624056,80.476467,53.654852
3,"3,999 and up",77.070764,80.928365,66.468891,81.106091,53.715094


In [17]:
# scores by school type

#figure out 4 appropriate ranges for per student budget using quartiles
type_group = students_grouped_by_school_pd

# need to do a weighted average (multiply the number of students 
# by the average math/reading scores then sum and divide by the quartile total)
type_group['school_total_math_scores'] = type_group['Total Students'] * type_group['Average Math Score']
type_group['school_total_reading_scores'] = type_group['Total Students'] * type_group['Average Reading Score']

# size_quartiles
# #re-sum the math and reading and retest the overall pass (same way as previous section)
type_group = type_group.groupby(['School Type']).agg({'Total Students': ['sum'],'passed_math': ['sum'], 'passed_reading': ['sum'], 'passed_overall': ['sum'], 'school_total_math_scores': ['sum'],'school_total_reading_scores': ['sum']}).reset_index()
type_group.columns = type_group.columns.droplevel(1)

type_group= type_group[['School Type','Total Students','passed_math','passed_reading','passed_overall','school_total_math_scores','school_total_reading_scores']]
type_group


type_group['Average Math Score'] = type_group['school_total_math_scores'] / type_group['Total Students']
type_group['Average Reading Score'] = type_group['school_total_reading_scores'] / type_group['Total Students']
type_group['% Passing Math'] = 100 * type_group['passed_math'] / type_group['Total Students']
type_group['% Passing Reading'] = 100 * type_group['passed_reading'] / type_group['Total Students']
type_group['% Overall Passing'] = 100 * type_group['passed_overall'] / type_group['Total Students']

type_output = type_group[['School Type','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing']]

type_output

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Charter,83.406183,83.902821,93.701821,96.645891,90.560932
1,District,76.987026,80.962485,66.518387,80.905249,53.695878
